In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn import tree
from sklearn import preprocessing

In [11]:
train_set.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1,1112094756,4rNk8PM,2018-04-14 10:06:05,2018-04-14 10:20:38,3,Universitario,En Curso,FEM,28,Representante de Atencion al Cliente/ Banco,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Call Center,RANDSTAD
1,1,1112334791,vVe1KDQ,2018-02-20 08:37:18,NaT,1,Secundario,Graduado,MASC,26,Cajero/a - Part Time,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,Farmacity
2,0,1112296457,akjGq0D,NaT,NaT,0,Secundario,Abandonado,MASC,45,OPERARIO DE PRODUCCIÓN,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,Adecco - Región NORTE & OESTE GBA
3,0,1112322128,owz3QPa,NaT,NaT,0,Secundario,Graduado,MASC,27,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1112317075,8MZ9LGj,2018-02-06 17:08:43,NaT,1,Universitario,Graduado,FEM,27,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
train_sample = train_set.sample(frac=0.7)
train_sample_classes = train_sample['sepostulo']

In [38]:
train_sample.drop(columns=['sepostulo', 'idaviso', 'idpostulante','titulo', 'denominacion_empresa'], inplace=True)
train_sample.drop(columns=['fechapostulacion', 'ultima_vista'], inplace=True)

In [39]:
train_sample.head()

,veces_visto,estudios,estado,sexo,edad,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
9275680,0,Terciario/Técnico,Graduado,FEM,24,NaN,NaN,NaN,NaN
9507944,1,Terciario/Técnico,Graduado,MASC,34,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas
5499227,0,Terciario/Técnico,Graduado,FEM,25,NaN,NaN,NaN,NaN
5389629,1,Universitario,Graduado,FEM,30,NaN,NaN,NaN,NaN
7744358,0,Universitario,Graduado,FEM,25,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Atención al Cliente


In [40]:
for column in train_sample.columns:
    if train_sample[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        train_sample[column] = le.fit_transform(train_sample[column].astype(str))

In [41]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_sample, train_sample_classes)

In [42]:
clf.score(train_sample, train_sample_classes)

0.9994398537714422

In [43]:
train_set = train_set.drop(train_sample.index)
test_classes= train_set['sepostulo']
train_set.drop(columns=['sepostulo'], inplace=True)

In [45]:
train_set.drop(columns=['idaviso', 'idpostulante','titulo', 'denominacion_empresa'], inplace=True)
train_set.drop(columns=['fechapostulacion', 'ultima_vista'], inplace=True)

In [50]:
for column in train_set.columns:
    if train_set[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        train_set[column] = le.fit_transform(train_set[column].astype(str))

In [51]:
clf.score(train_set, test_classes)

0.9991232776595299

In [55]:
test100k = pd.read_csv('test_final_100k_complete.csv', low_memory=False)

In [62]:
for column in test100k.columns:
    if test100k[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        test100k[column] = le.fit_transform(test100k[column].astype(str))

In [64]:
results=clf.predict(test100k)

In [65]:
results

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [77]:
results.mean()

0.34088

In [71]:
test100k=test100k.reset_index(drop=False)

In [68]:
test100k['sepostulo']=0

In [73]:
resultado_100k=test100k[['index','sepostulo']]

In [75]:
resultado_100k.rename(columns={'index':'id'}, inplace=True)

/home/maximiliano/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [78]:
resultado_100k['sepostulo']=results

/home/maximiliano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
resultado_100k.to_csv('100k_decisiontree1.csv', index=False)